In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
from detection_service.model_loader import *
from detection_service.postprocess import *
from detection_service.preprocess import *
from detection_service.visualize import *

In [ ]:
conf_thres: float =.4
iou_thres: float =.45
max_det:int =  1000
agnostic_nms: bool = False
onnx_path = "../detection_service/yolov6s_face.onnx"
img_list = [cv2.imread('../example-data/image1.jpg'),
           cv2.imread('../example-data/image2.jpg'),
           cv2.imread('../example-data/image3.jpg')]

session = create_session(onnx_path)

In [ ]:
origin_imgs, prepared_data = preprocess_images(img_list)
np_batch = np.concatenate([data[0] for data in prepared_data])
im = np.ascontiguousarray(np_batch/255)

print(f"[INFO] Run inference on batch shape: {np_batch.shape}")
out = session.run(None, {'images':im})[0]
dets = non_max_suppression_face(torch.Tensor(out), conf_thres, iou_thres, [i for i in range(80)], agnostic_nms, max_det=max_det)

In [ ]:
rescaled_detections = rescale_detections(dets, prepared_data)

def show_out(origin_image, detections):
    for detection in detections:
        label = f"{detection["conf"]:.2f}"
        plot_box_and_label(origin_image, max(round(sum(origin_image.shape) / 2 * 0.003), 2), detection, label)
    import matplotlib.pyplot as plt
    plt.figure(figsize=(20, 30))
    plt.imshow(origin_image)

ind = 1
show_out(origin_imgs[ind].copy(), rescaled_detections[ind])